# Worksheet 08

Name:  Zhiyi Yu
UID: U65310997

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[2.890575222726332, 6.13118819643281, 5.14977839004515, 5.91914913708844, 3.214294359571733, 3.968394415770595, 7.353433134764183, 6.6067393942242045, 2.7230091825982554, 5.3791726334992305]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)


[9.282235166151182, 7.466749510536794, 7.039678783615328, 8.811452379823079, 7.887902963532992, 7.7319573206542, 8.429967724290215, 7.96986809691679, 7.96992127768984, 7.466318051838139]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [5]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.3791726334992305, 2.7230091825982554, 7.466318051838139, 7.96992127768984, 7.96986809691679, 8.429967724290215, 7.7319573206542, 6.6067393942242045, 7.887902963532992, 8.811452379823079]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mean for each component, covariance for each component, mixture weight for each component

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [6]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]
mean = [sum(s1)/len(s1), sum(s2)/len(s2)]
var = [sum(map(lambda x: (x - mean[0])**2, s1)) / len(s1),
       sum(map(lambda x: (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.3791726334992305, 2.7230091825982554]
[7.466318051838139, 7.96992127768984, 7.96986809691679, 8.429967724290215, 7.7319573206542, 6.6067393942242045, 7.887902963532992, 8.811452379823079]
P(S_1) = 0.2,  P(S_2) = 0.8
mean_1 = 4.051090908048743,  mean_2 = 7.859265901121184
var_1 = 1.7638010694755442,  var_2 = 0.3746369172511683


The estimated parameters from KMeans clustering have some differences from the true values. These discrepancies can be attributed to the randomness in data generation, the specific method by which KMeans determines clusters, and the sample size. The estimated means are reasonably close, but the variances show some deviation. 

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [7]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5)) # using standard deviation which is sqrt(variance)
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_i = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x_i)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x_i)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.3791726334992305
probability of observing that point if it came from cluster 0 =  0.18219565850668304
probability of observing that point if it came from cluster 1 =  0.0001773938891642814
point =  2.7230091825982554
probability of observing that point if it came from cluster 0 =  0.18219565850668298
probability of observing that point if it came from cluster 1 =  3.3347169517624406e-16
point =  7.466318051838139
probability of observing that point if it came from cluster 0 =  0.011008296319696232
probability of observing that point if it came from cluster 1 =  0.5304039253413589
point =  7.96992127768984
probability of observing that point if it came from cluster 0 =  0.003863731563715324
probability of observing that point if it came from cluster 1 =  0.6412207264455784
point =  7.96986809691679
probability of observing that point if it came from cluster 0 =  0.003864188116678795
probability of observing that point if it came from cluster 1 =  0.6412307962998166
point =  8

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [8]:
prob_c = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]


mean = [
    sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
    sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)
]


var = [
    sum([prob_s0_x[i] * (data[i] - mean[0]) ** 2 for i in range(len(data))]) / sum(prob_s0_x),
    sum([prob_s1_x[i] * (data[i] - mean[1]) ** 2 for i in range(len(data))]) / sum(prob_s1_x)
]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.21380021625391513,  P(S_2) = 0.786199783746085
mean_1 = 4.225969061567387,  mean_2 = 7.878554448015107
var_1 = 2.0987931826214155,  var_2 = 0.358115653718746


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [9]:
updated_prob_s0_x = [] # Updated P(S_0 | X_i)
updated_prob_s1_x = [] # Updated P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    updated_prob_s0_x.append(pdf_i[0] * prob_c[0] / prob_x_i)
    updated_prob_s1_x.append(pdf_i[1] * prob_c[1] / prob_x_i)

updated_probs = zip(data, updated_prob_s0_x, updated_prob_s1_x)
for p in updated_probs:
    print(p[0])
    print("Updated Probability of coming from S_1 = " + str(p[1]))
    print("Updated Probability of coming from S_2 = " + str(p[2]))
    print()

differences_s0 = [abs(a - b) for a, b in zip(prob_s0_x, updated_prob_s0_x)]
differences_s1 = [abs(a - b) for a, b in zip(prob_s1_x, updated_prob_s1_x)]
print("Average difference in P(S_1 | X_i):", sum(differences_s0) / len(differences_s0))
print("Average difference in P(S_2 | X_i):", sum(differences_s1) / len(differences_s1))


5.3791726334992305
Updated Probability of coming from S_1 = 0.9980123182110998
Updated Probability of coming from S_2 = 0.0019876817889001638

2.7230091825982554
Updated Probability of coming from S_1 = 0.9999999999999988
Updated Probability of coming from S_2 = 1.1650835119861967e-15

7.466318051838139
Updated Probability of coming from S_1 = 0.011538775656167963
Updated Probability of coming from S_2 = 0.988461224343832

7.96992127768984
Updated Probability of coming from S_1 = 0.004013832167540448
Updated Probability of coming from S_2 = 0.9959861678324596

7.96986809691679
Updated Probability of coming from S_1 = 0.0040141572040926134
Updated Probability of coming from S_2 = 0.9959858427959073

8.429967724290215
Updated Probability of coming from S_1 = 0.0025421551858988665
Updated Probability of coming from S_2 = 0.997457844814101

7.7319573206542
Updated Probability of coming from S_1 = 0.0061530640560941785
Updated Probability of coming from S_2 = 0.9938469359439059

6.606739394

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [11]:
hard_assignments = [0 if prob_s0 > prob_s1 else 1 for prob_s0, prob_s1 in zip(updated_prob_s0_x, updated_prob_s1_x)]

assigned_data = list(zip(data, hard_assignments))

for point, cluster in assigned_data:
    print(f"Data Point: {point}  Cluster: {cluster}")


Data Point: 5.3791726334992305  Cluster: 0
Data Point: 2.7230091825982554  Cluster: 0
Data Point: 7.466318051838139  Cluster: 1
Data Point: 7.96992127768984  Cluster: 1
Data Point: 7.96986809691679  Cluster: 1
Data Point: 8.429967724290215  Cluster: 1
Data Point: 7.7319573206542  Cluster: 1
Data Point: 6.6067393942242045  Cluster: 1
Data Point: 7.887902963532992  Cluster: 1
Data Point: 8.811452379823079  Cluster: 1
